In [3]:
from pykiwoom.kiwoom import *

kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

kospi = kiwoom.GetCodeListByMarket('0')
kosdaq = kiwoom.GetCodeListByMarket('10')
etf = kiwoom.GetCodeListByMarket('8')

print(len(kospi), kospi)
print(len(kosdaq), kosdaq)
print(len(etf), etf)

AttributeError: 'QAxWidget' object has no attribute 'OnReceiveTrData'

In [64]:
import re
import pandas as pd
import requests
import FinanceDataReader as fdr
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [166]:
#특정 날짜 23년 2월 2일 크롤링
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23&wr_id=2190#section1'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml')    

In [167]:
#특정 날짜 23년 2월 2일 크롤링
df = pd.DataFrame(columns=['comment'])
for i in range(len(bs.select('.cmt_contents'))):
    comment = bs.select('.cmt_contents')[i].text
    df.loc[i] = [comment]

In [168]:
df

,comment
0,\n\n \n
1,\n\n \n
2,\n\n \n
3,\n\n \n
4,\n\n전일 미국증시부터 살펴보고 가겠습니다. \n
...,...
81,\n\n\n
82,\n\n\n
83,\n\n \n
84,"\n\n코스피 상승, 코스닥 상승 거래소 : 개인 매도 / 외국인 매수 / 기관 매..."


In [4]:
#게시판에서 각 게시물 주소얻기 
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml') 

In [198]:
len(bs.select('table>tr'))

33

In [200]:
#1페이지 링크 얻기
for i in range(len(bs.select('table>tr'))):
    date = bs.select('table>tr')[i].select('dd')[1].text
    url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [5]:
#해당 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
for page_num in range(1,4):
    url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&page={page_num}'
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
        )
    bs = BeautifulSoup(response.text, 'lxml') 

    for i in range(len(bs.select('table>tr'))):
        date = bs.select('table>tr')[i].select('dd')[1].text
        url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [8]:
def get_info_from_url(url):
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
    )
    bs = BeautifulSoup(response.text, 'lxml') 
    return bs

In [9]:
#모든 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
df = pd.DataFrame()
for year in tqdm(range(19,24)):
    for page_num in tqdm(range(1,4)):
        url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list{year}&page={page_num}'
        bs = get_info_from_url(url)  
        info_from_bs = bs.select('table>tr')
        for bs in info_from_bs:
            try :
                date = bs.select('dd')[1].text
                url_link = bs.select('a')[0].get('href')
                bs = get_info_from_url(url_link)
                data_from_url = bs.select('.cmt_contents')

                for comments in data_from_url:
                    comment = comments.text

                    temp = pd.DataFrame({
                        'date': date,
                        'url': url_link,
                        'comment': comment
                        }, index=[0])

                    df = pd.concat([df, temp], axis=0)
            except IndexError:
                print(1)

100%|██████████| 5/5 [02:36<00:00, 31.31s/it]

1
1


In [41]:
#파일로 저장
df.to_csv('data.csv', index=False, encoding='utf-8-sig')

In [82]:
#파일 불러오기
df = pd.read_csv('./data.csv')

In [73]:
#필터링 연습
#매수 포함 가져오기
df = df[df['comment'].str.contains('매수', na =False)]
#현재시간 포함 가져오기
df = df[df['comment'].str.contains('현재시간', na =False)]
df = df[df['comment'].str.contains('▶', na =False)]
#공백 지우기
df['comment']=df.comment.apply(lambda x : re.sub('\\n',' ',x) )


In [83]:
#금일 추천주 필터링
df = df[df['comment'].str.contains('매수', na =False)]
df = df[df['comment'].str.contains('현재시간', na =False)]
df = df[df['comment'].str.contains('▶', na =False)]
df['comment'] = df.comment.apply(lambda x : re.sub('\\n','',x))

In [84]:
#date 를 datetime 형식으로 변환 1
df['date'] = df.date.apply(lambda x : x.replace(".",""))
df['date']= df['date'].astype('str')
df['date'] = df.date.apply(lambda x : datetime.strptime(x,'%Y%m%d'))
df = df.reset_index(drop=True)

In [86]:
#date 를 datetime 형식으로 변환 2
df2 = df 
df2['date'] = pd.to_datetime(df2['date'])

In [81]:
# 예외찾기
df2 = df['slicing_comment'].apply(lambda x : x == [] )
df2[df2]

KeyError: 'slicing_comment'

In [85]:
#예외처리
df.comment[54] = df.comment[54].replace('\xa0',' ')

C:\Users\khlee\AppData\Local\Temp\ipykernel_12756\3069763651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.comment[54] = df.comment[54].replace('\xa0',' ')


In [86]:
#종목 슬라이스
df['slicing_comment'] = df.comment.apply(lambda x : re.findall('\d+분 ([가-힣A-z]+)',x)[0])

In [87]:
#종목명 추출
recommended_stock = df.filter(items=['date','slicing_comment'])
recommended_stock = recommended_stock.rename(columns={'slicing_comment':'stock_name'})

In [112]:
#데이터프레임 한번에 보기
pd.set_option('display.max_rows', None)
#다시 돌리기
pd.options.display.max_rows = 100

In [54]:
#증권거래 정보 가져오기
df_krx = fdr.StockListing('KRX')
df_krx = df_krx.filter(items = ['Code','Name'])
df_krx = df_krx.rename(columns={'Name':'stock_name'})

In [97]:
df_krx2 = fdr.StockListing('KRX')

In [100]:
df_krx2.query('Code == "006980"')

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
1881,006980,KR7006980007,우성,KOSPI,,23700,2,-250,-1.04,23600,23900,23350,5942,139908350,73233000000,3090000,STK


In [92]:
stock = pd.merge(recommended_stock, df_krx, how = 'left', on  = 'stock_name')

In [88]:
recommended_stock

,date,stock_name
0,2019-12-27,에스맥
1,2019-12-27,화천기계
2,2019-12-27,루미마이크로
3,2019-12-26,한국화장품
4,2019-12-26,디앤씨미디어
...,...,...
967,2022-12-22,팬오션
968,2022-12-21,세종메디칼
969,2022-12-21,청담글로벌
970,2022-12-19,노터스


In [96]:
# 예외찾기
stock[stock['Code'].isnull()]


,date,stock_name,Code
2,2019-12-27,루미마이크로,NaN
6,2019-12-26,제낙스,NaN
9,2019-12-24,에스모머티리얼즈,NaN
11,2019-12-23,우성사료,NaN
15,2019-12-20,에스모,NaN
...,...,...,...
865,2022-01-19,동국S,NaN
892,2022-01-04,나노스,NaN
932,2023-02-06,KGETS,NaN
945,2023-01-18,yg,NaN
